Do Pokémon with names containing aggressive phonological signals (e.g., harsh consonants, longer syllable length, or suffixes like “-gon,” “-don,” “-zar”) exhibit higher Attack and Speed stats and greater total base stats compared to those without such signals?

Null Hypothesis (H₀)

There is no significant difference in Attack, Speed, or total base stats between Pokémon whose names contain aggressive phonological signals and those whose names do not.

Alternative Hypothesis (H₁)

Pokémon whose names contain aggressive phonological signals have significantly higher Attack, Speed, and total base stats compared to those without such signals.

In [ ]:
import requests
import pandas as pd
import re
from scipy.stats import ttest_ind, mannwhitneyu

BASE = "https://pokeapi.co/api/v2"
SESSION = requests.Session()

def fetch_json(url):
    resp = SESSION.get(url, timeout=30)
    resp.raise_for_status()
    return resp.json()

def flatten_stats(stats_list):
    out = {k: None for k in [
        "hp","attack","defense","special-attack","special-defense","speed"
    ]}
    for s in stats_list:
        key = s.get("stat", {}).get("name")
        val = s.get("base_stat")
        if key in out:
            out[key] = val
    return out

def flatten_types(types_list):
    sorted_types = sorted(types_list, key=lambda t: t.get("slot", 99))
    return [t.get("type", {}).get("name") for t in sorted_types if t.get("type")]


In [ ]:
# Get Pokémon index
index = fetch_json(f"{BASE}/pokemon?limit=20000&offset=0")
results = index.get("results", [])
print(f"Found {len(results)} Pokémon entries")


Found 1302 Pokémon entries


In [ ]:
# Fetch detailed data
rows = []
for i, entry in enumerate(results, start=1):
    data = fetch_json(entry["url"])
    stats = flatten_stats(data.get("stats", []))
    types = flatten_types(data.get("types", []))

    rows.append({
        "id": data.get("id"),
        "name": data.get("name"),
        "height_dm": data.get("height"),
        "weight_hg": data.get("weight"),
        "types": types,
        "num_types": len([t for t in types if t]),
        "hp": stats["hp"],
        "attack": stats["attack"],
        "defense": stats["defense"],
        "special_attack": stats["special-attack"],
        "special_defense": stats["special-defense"],
        "speed": stats["speed"],
        "total_stats": sum(v for v in stats.values() if isinstance(v, (int, float)))
    })

    if i % 100 == 0:
        print(f"Fetched {i} Pokémon")

Fetched 100 Pokémon
Fetched 200 Pokémon
Fetched 300 Pokémon
Fetched 400 Pokémon
Fetched 500 Pokémon
Fetched 600 Pokémon
Fetched 700 Pokémon
Fetched 800 Pokémon
Fetched 900 Pokémon
Fetched 1000 Pokémon
Fetched 1100 Pokémon
Fetched 1200 Pokémon
Fetched 1300 Pokémon


In [ ]:
# Clean & save
df = pd.DataFrame(rows)
df = df.dropna(subset=["name", "attack", "speed"]).reset_index(drop=True)

def pick(lst, idx):
    try:
        return lst[idx]
    except:
        return None

df["type_1"] = df["types"].apply(lambda xs: pick(xs, 0))
df["type_2"] = df["types"].apply(lambda xs: pick(xs, 1))

df.to_csv("pokemon_base_stats.csv", index=False)
print(" Saved pokemon_base_stats.csv with", len(df), "rows.")

 Saved pokemon_base_stats.csv with 1302 rows.


Feature Engineering

In [ ]:
# Load dataset
df = pd.read_csv("pokemon_base_stats.csv")

# 1. Phonological feature definitions

HARD_LETTERS = set(list("kgxz"))  # strong consonants
AGGR_CLUSTER_RE = re.compile(r"(gr|kr|dr|tr|br|pr|rk|rg|sk|st|gz|kk|gg|xx|zz)", re.IGNORECASE)
AGGR_SUFFIXES = ("don","gon","dra","tar","gar","rex","zar")
SOFT_SUFFIXES = ("ee","chu","nyan","puff","bell","ly","lly","py","pichu")

VOWELS = "aeiouy"

# 2. Helper functions

def count_syllables(name: str) -> int:
    n = name.lower()
    syll = 0
    prev_v = False
    for ch in n:
        is_v = (ch in VOWELS)
        if is_v and not prev_v:
            syll += 1
        prev_v = is_v
    return max(1, syll)

def count_hard_letters(name: str) -> int:
    return sum(ch in HARD_LETTERS for ch in name.lower())

def count_aggr_clusters(name: str) -> int:
    return len(AGGR_CLUSTER_RE.findall(name))

def has_aggr_suffix(name: str) -> bool:
    return name.lower().endswith(AGGR_SUFFIXES)

def has_soft_suffix(name: str) -> bool:
    return name.lower().endswith(SOFT_SUFFIXES)



In [ ]:
# 3. Compute phonological features

df["syllables"] = df["name"].apply(count_syllables)
df["hard_count"] = df["name"].apply(count_hard_letters)
df["cluster_count"] = df["name"].apply(count_aggr_clusters)
df["aggr_suffix"] = df["name"].apply(has_aggr_suffix)
df["soft_suffix"] = df["name"].apply(has_soft_suffix)

In [ ]:
# 4. Score system

# Base scoring weights
# Clusters = strong signal (weight 3)
# Aggressive suffix = strong (weight 3)
# Hard letters (capped at 3) = medium (weight 1)
# Long syllables (≥4) = light (weight 1)
# Soft suffix = penalty (-3)
df["aggr_score"] = (
    (df["cluster_count"] * 3) +
    (df["aggr_suffix"].astype(int) * 3) +
    (df["hard_count"].clip(upper=3) * 1) +
    ((df["syllables"] >= 4).astype(int) * 1) -
    (df["soft_suffix"].astype(int) * 3)
)

# Ensure no negative scores
df["aggr_score"] = df["aggr_score"].clip(lower=0)

In [ ]:
# 5. Find best threshold for balance

scores = sorted(df["aggr_score"].unique())
n = len(df)
best_thr, best_diff, best_rate = None, 999, None

for thr in scores:
    rate = (df["aggr_score"] >= thr).mean()
    diff = abs(rate - 0.5)  # try to split 50/50
    if diff < best_diff:
        best_thr, best_diff, best_rate = thr, diff, rate

df["aggressive_name_flag"] = (df["aggr_score"] >= best_thr).astype(int)

print(f"Best threshold: {best_thr}")
print(f"Best Rate: {best_rate:.2%}")
print(df["aggressive_name_flag"].value_counts())


Best threshold: 2
Best Rate: 41.09%
aggressive_name_flag
0    767
1    535
Name: count, dtype: int64


In [ ]:
# 6. False positives & top scores

# Check suspicious soft-sounding aggressive labels
df["potential_false_positive"] = (df["aggressive_name_flag"] == 1) & (df["soft_suffix"])
fp_rate = df["potential_false_positive"].mean()
print(f"Potential false positive rate: {fp_rate:.2%}")

print("\n Top aggressive names by score:")
print(df.sort_values("aggr_score", ascending=False).head(15)[["name","aggr_score"]])

print("\n Example suspicious aggressive labels:")
print(df[df["potential_false_positive"]].head(15)[["name","aggr_score"]])

Potential false positive rate: 0.23%

 Top aggressive names by score:
                          name  aggr_score
1055           gourgeist-large          13
710          gourgeist-average          10
1297   ogerpon-wellspring-mask          10
1299  ogerpon-cornerstone-mask           9
1101             kyogre-primal           9
1204            stunfisk-galar           9
1056           gourgeist-super           9
1104         pikachu-rock-star           9
910                 skeledirge           9
983                 great-tusk           8
1054           gourgeist-small           8
229                    kingdra           8
816                   drizzile           8
1106          pikachu-pop-star           8
1102            groudon-primal           8

 Example suspicious aggressive labels:
                  name  aggr_score
38          jigglypuff           2
795          xurkitree           5
1282  tatsugiri-droopy           2


In [ ]:
# 7. Save engineered features

df.to_csv("pokemon_phonology_features_final.csv", index=False)
print(" Saved pokemon_phonology_features_final.csv with engineered features.")

 Saved pokemon_phonology_features_final.csv with engineered features.


Hypothesis Testing

In [ ]:
# Load dataset
df = pd.read_csv("pokemon_phonology_features_final.csv")

# Split groups
aggressive = df[df["aggressive_name_flag"] == 1]
non_aggressive = df[df["aggressive_name_flag"] == 0]

# Variables to test
features = ["attack", "speed", "total_stats"]


In [ ]:
# T-test
print(" T-TEST RESULTS")
alpha = 0.05  # significance level
for feature in features:
    t_stat, p_value = ttest_ind(aggressive[feature], non_aggressive[feature], equal_var=False)
    mean_aggr = aggressive[feature].mean()
    mean_non = non_aggressive[feature].mean()
    print(f"\nFeature: {feature.upper()}")
    print(f"Mean (Aggressive) = {mean_aggr:.2f}")
    print(f"Mean (Non-Aggressive) = {mean_non:.2f}")
    print(f"T-statistic = {t_stat:.3f}, p-value = {p_value:.5f}")
    if p_value < alpha:
        if mean_aggr > mean_non:
            print("Reject H₀ = Aggressive names have significantly HIGHER stats.")
        else:
            print("Reject H₀ = Aggressive names have significantly LOWER stats.")
    else:
        print("Fail to reject H₀ = No significant difference.")
print("There is enough evidence to say that aggressive names have significantly Higher stats.")

 T-TEST RESULTS

Feature: ATTACK
Mean (Aggressive) = 90.14
Mean (Non-Aggressive) = 75.62
T-statistic = 8.245, p-value = 0.00000
Reject H₀ = Aggressive names have significantly HIGHER stats.

Feature: SPEED
Mean (Aggressive) = 75.93
Mean (Non-Aggressive) = 67.83
T-statistic = 4.715, p-value = 0.00000
Reject H₀ = Aggressive names have significantly HIGHER stats.

Feature: TOTAL_STATS
Mean (Aggressive) = 481.17
Mean (Non-Aggressive) = 421.36
T-statistic = 9.079, p-value = 0.00000
Reject H₀ = Aggressive names have significantly HIGHER stats.
There is enough evidence to say that aggressive names have significantly Higher stats.


In [ ]:
# Load the final dataset
df = pd.read_csv("pokemon_phonology_features_final.csv")

# Count how many are flagged as false positives
fp_count = df["potential_false_positive"].sum()
tp_count = df["aggressive_name_flag"].sum()
fp_rate = fp_count / tp_count if tp_count > 0 else 0

print(f"Potential false positives: {fp_count} out of {tp_count} aggressive names")
print(f"False positive rate ≈ {fp_rate:.2%}")
print("\n Suspicious aggressive labels:")
print(df[df["potential_false_positive"]].head(20)[["name","aggr_score","aggressive_name_flag"]])


Potential false positives: 3 out of 535 aggressive names
False positive rate ≈ 0.56%

 Suspicious aggressive labels:
                  name  aggr_score  aggressive_name_flag
38          jigglypuff           2                     1
795          xurkitree           5                     1
1282  tatsugiri-droopy           2                     1
